here we make the api calls for the data

In [1]:
import requests
import pandas as pd
import time

API_KEY = "CG-YbJf36XgKvEuq3n5Sv4fbXgC"
HEADERS = {
    "x-cg-pro-api-key": API_KEY
}

In [2]:
def get_ohlc_data(coin_id, vs_currency='usd', days='max'):
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/ohlc"
    params = {
        "vs_currency": vs_currency,
        "days": days  # 'max' gives the full history
    }
    response = requests.get(url, headers=HEADERS, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df[["date", "open", "high", "low", "close"]]


In [3]:
def get_market_chart(coin_id, vs_currency='usd', days='max'):
    """
    Retrieves daily price, market cap, and volume data for a given cryptocurrency.

    Args:
        coin_id (str): The CoinGecko coin ID (e.g., 'bitcoin').
        vs_currency (str): The fiat currency to compare against (default: 'usd').
        days (str): Duration of data to fetch (default: 'max' for full history).

    Returns:
        pd.DataFrame: DataFrame containing date, price, market_cap, and volume.
    """
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        "vs_currency": vs_currency,
        "days": days,
        "interval": "daily"
    }

    response = requests.get(url, headers=HEADERS, params=params)
    response.raise_for_status()
    data = response.json()

    # Convert and align timestamps
    prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
    market_caps = pd.DataFrame(data["market_caps"], columns=["timestamp", "market_cap"])
    volumes = pd.DataFrame(data["total_volumes"], columns=["timestamp", "volume"])

    for df in (prices, market_caps, volumes):
        df["date"] = pd.to_datetime(df["timestamp"], unit="ms")

    # Merge into one dataframe
    merged = prices[["date", "price"]].merge(
        market_caps[["date", "market_cap"]], on="date"
    ).merge(
        volumes[["date", "volume"]], on="date"
    )

    return merged




In [4]:
def get_full_data(coin_id):
    market = get_market_chart(coin_id)
    return market


In [5]:
btc_data = get_full_data("bitcoin")
xtz_data = get_full_data("tezos")

btc_data.to_csv("bitcoin_data.csv", index=False)
xtz_data.to_csv("tezos_data.csv", index=False)
